In [1]:
from sqlite3 import connect
import datetime as dt
from astropy.coordinates import SkyCoord, Angle
from astropy.time import Time
import astropy.units as u
import  astroplan as ap
import pandas as pd


In [ ]:
def all_night_observing_session(observer: ap.Observer, utc_date: str) -> [Time, Time]:
    return [observer.twilight_evening_civil(utc_date, which='nearest'), observer.twilight_morning_civil(utc_date, which='next')]

In [139]:
conn = connect("../django/QuadEBdb/db.sqlite3")

# read base list of targets
targets = pd.read_sql(
    """
        select *
        from tom_target t
        join tom_catalogassociation ca on ca.target_id = t.id and ca.catalog = 'TESS TICv8' and ca.association = 'Primary ID'
        join tom_tess_ticv8 tt on tt.Identifier = ca.catalog_id
        ;""",
    conn,
    index_col="target_id",
)

# get speckle and spectrum data and add it to main targets table
speckle = pd.read_sql("select * from tom_specklerawdata;", conn, index_col="target_id")
spectrum = pd.read_sql(
    "select * from tom_spectrumrawdata;", conn, index_col="target_id"
)

targets = targets.join(speckle.groupby("target_id").id.agg(num_speckle="count")).rename(
    columns={"num_speckle": "Num Speckle"}
)
targets = targets.join(
    spectrum.groupby("target_id").id.agg(num_spectra="count")
).rename(columns={"num_spectra": "Num Spectra"})
targets.fillna(0, inplace=True)
targets["Num Speckle"] = targets["Num Speckle"].astype(int)
targets["Num Spectra"] = targets["Num Spectra"].astype(int)

# add columns for membership in each target list
for (target_list,) in conn.execute("select name from tom_targetlist;").fetchall():
    list_members = [
        result[0]
        for result in conn.execute(
            """
                                select t.local_id
                                from tom_targetlist tl
                                join tom_targetlist_targets tlt on tlt.targetlist_id = tl.id
                                join tom_target t on t.id = tlt.target_id
                                where tl.name = ?
                                ;
                                """,
            [target_list],
        ).fetchall()
    ]
    targets[target_list] = False
    targets.loc[targets.local_id.isin(list_members), target_list] = True

# add parameters for the binary systems
params = pd.read_sql(
    """
    select t.local_id, bp.*
    from tom_binaryparameters bp
    join tom_scienceresult sr on sr.id = bp.scienceresult_ptr_id
    join tom_target t on t.id = sr.target_id
    ;""",
    conn,
    index_col="local_id",
)
# TODO: following code only works for binary systems - generalize to n-member systems
joined_params = params.drop(columns=["scienceresult_ptr_id"])
joined_params = joined_params[joined_params["member"] == "Member A"].join(
    joined_params[joined_params["member"] == "Member B"],
    on="local_id",
    lsuffix="_A",
    rsuffix="_B",
)
targets = targets.join(joined_params, on="local_id", how="left")

targets

,id,local_id,source,target_type,id,catalog,catalog_id,association,id,version,...,depth_secondary_A,duration_primary_A,member_B,period_B,t0_primary_B,t0_secondary_B,duration_secondary_B,depth_primary_B,depth_secondary_B,duration_primary_B
target_id,,,,,,,,,,,,,,,,,,,,,
3366,3366,TIC 270360534,Kostov 2023 arXiv:2309.14200,QuadEB,1024,TESS TICv8,270360534,Primary ID,236,20190415,...,69.0,14.0,Member B,-999.000000,2024.880100,1989.245900,11.4,NaN,NaN,9.2
3367,3367,TIC 219469945,Kostov 2022 arXiv:2202.05790,QuadEB,1025,TESS TICv8,219469945,Primary ID,323,20190415,...,43.0,4.1,Member B,14.965529,1957.709900,1964.924782,7.3,80.0,20.0,6.3
3368,3368,TIC 20212631,Kostov 2023 arXiv:2309.14200,QuadEB,1026,TESS TICv8,20212631,Primary ID,310,20190415,...,NaN,1.2,Member B,9.478670,1933.141800,NaN,NaN,76.0,NaN,4.4
3369,3369,TIC 150055835,Kostov 2023 arXiv:2309.14200,QuadEB,1027,TESS TICv8,150055835,Primary ID,389,20190415,...,81.0,5.0,Member B,13.784160,2477.580800,1823.679700,6.7,41.0,23.0,7.0
3370,3370,TIC 161043618,Kostov 2022 arXiv:2202.05790,QuadEB,1028,TESS TICv8,161043618,Primary ID,215,20190415,...,20.0,4.0,Member B,1.488497,1739.159400,1739.907965,2.5,50.0,20.0,2.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3559,3559,TIC 414969157,Kostov 2022 arXiv:2202.05790,QuadEB,1217,TESS TICv8,414969157,Primary ID,298,20190415,...,120.0,3.3,Member B,6.928951,1880.071700,1883.340779,3.1,110.0,50.0,3.8
3560,3560,TIC 27543409,Kostov 2022 arXiv:2202.05790,QuadEB,1218,TESS TICv8,27543409,Primary ID,387,20190415,...,15.0,NaN,Member B,4.013356,1494.513000,1494.513000,NaN,75.0,NaN,NaN
3561,3561,TIC 139914081,Kostov 2023 arXiv:2309.14200,QuadEB,1219,TESS TICv8,139914081,Primary ID,235,20190415,...,NaN,1.2,Member B,15.796390,1333.657700,1328.352800,3.7,215.0,30.0,5.3


In [160]:
#make astroplan objects for each target
fixed_targets = [
    ap.FixedTarget(
        coord=SkyCoord(
            frame="icrs",
            obstime=Time("2000.0", format="jyear", scale="tdb"),
            ra=target["ra"] * u.deg,
            dec=target["dec"] * u.deg,
            pm_ra_cosdec=target["pmRA"] * u.mas / u.yr,
            pm_dec=target["pmDEC"] * u.mas / u.yr,
            # distance=target["plx"] * u.pc,
        ),
        name=target["local_id"],
    )
    for (_, target) in targets.iterrows()
]

observer = ap.Observer.at_site("lbt")

constraints = [
    ap.AltitudeConstraint(30*u.deg, 80*u.deg),
    ap.AirmassConstraint(2),
    # ap.AtNightConstraint.twilight_civil(),
    ]

observing_nights_utc = [
    # '2024-01-13',
    # '2024-01-14',
    # '2024-01-15',
    # '2024-01-16',
    # '2024-01-17',
    # '2024-01-18',
    '2024-02-19',
]

#add a column to table for observability of each target
targets["Observable Nights"] = 0
for observing_night_utc in observing_nights_utc:
    night = Time(observing_night_utc)
    observing_session = [observer.twilight_evening_civil(night, which='nearest'), observer.twilight_morning_civil(night, which='next')]
    observable_tonight = ap.is_observable(constraints, observer, fixed_targets, observing_session)
    targets.loc[observable_tonight, "Observable Nights"] += 1


In [109]:
# criteria={
#     # "test": [(targets["HQND"]), ""],
#     "Other Bright with spectra and speckle": [(targets["magnitude"] < 11) & (targets["Num Speckle"] > 0) & (targets["Num Spectra"] > 0) & (targets["HQND"] == False), "Low"],
#     "HQND, Dim": [(targets["HQND"]) & (targets["magnitude"]>13), "Low"],
#     "HQND Bright no spectra": [(targets["HQND"]) & (targets["Num Spectra"] == 0) & (targets["magnitude"] < 13), "Medium"],
#     "HQND Bright with spectra": [(targets["HQND"]) & (targets["Num Spectra"] > 0) & (targets["magnitude"] < 13), "High"],
#     "Featured": [(targets["Featured targets"]), "Highest"],
# }

criteria = {
    "VATT Test": [(targets["period_A"] < 1) & (targets["period_B"] > 5), "Highest"],
}

matching_ids = {}
combined_ids = set()
targets["Priority"] = ""
for name, [criterion, priority] in criteria.items():
    filtered_ids = set(targets[criterion  & (targets["Observable Nights"] > 0)]["local_id"])
    targets.loc[targets.local_id.isin(filtered_ids), "Priority"]=priority
    print(f"{len(filtered_ids):3d} targets from criterion: {name}")
    combined_ids=combined_ids | filtered_ids
    matching_ids[name] = filtered_ids
print(f"Total of {len(combined_ids)} targets")

print()
for name,ids in matching_ids.items():
    print(f"Criteria {name}:")
    print(f"    {ids}")

  4 targets from criterion: VATT Test
Total of 4 targets

Criteria VATT Test:
    {'TIC 238558210', 'TIC 24700485', 'TIC 20212631', 'TIC 13021681'}


In [157]:
min_per, max_per = 1.5, 5
filtered_columns = [
    "local_id","source","Vmag","GAIAmag", "period_A","period_B","duration_primary_A","duration_secondary_A","duration_primary_B","duration_secondary_B","depth_primary_A","depth_primary_B",]
asd = targets[
    (
        (targets["period_A"] > max_per) & (targets["period_B"] < min_per) & (targets["period_B"] > 0)
        | (targets["period_A"] < min_per) & (targets["period_B"] > max_per) & (targets["period_A"] > 0)
    )
    & (targets["Observable Nights"] > 0)
].sort_values(by="GAIAmag")[filtered_columns]
print(len(asd))
asd

9


,local_id,source,Vmag,GAIAmag,period_A,period_B,duration_primary_A,duration_secondary_A,duration_primary_B,duration_secondary_B,depth_primary_A,depth_primary_B
target_id,,,,,,,,,,,,
3449,TIC 238558210,Kostov 2023 arXiv:2309.14200,9.370,9.25867,0.972387,5.337670,3.5,NaN,4.8,5.2,33.0,33.0
3385,TIC 470710327,Kostov 2022 arXiv:2202.05790,9.650,9.52218,1.104686,19.950922,5.8,5.5,NaN,NaN,60.0,70.0
3368,TIC 20212631,Kostov 2023 arXiv:2309.14200,10.580,10.51590,0.297159,9.478670,1.2,NaN,4.4,NaN,215.0,76.0
3466,TIC 283940788,Kostov 2022 arXiv:2202.05790,11.774,11.72660,0.876867,8.167894,6.5,6.5,6.6,8.5,30.0,165.0
3419,TIC 24700485,Kostov 2023 arXiv:2309.14200,12.783,12.52720,0.739810,10.672330,1.5,1.5,2.6,NaN,5.0,74.0
3397,TIC 375325607,Kostov 2022 arXiv:2202.05790,12.682,12.58840,1.311984,9.223201,4.8,3.5,6.9,8.6,45.0,75.0
3428,TIC 36439758,Kostov 2023 arXiv:2309.14200,13.115,13.02850,1.415610,8.213710,3.0,NaN,8.8,NaN,NaN,108.0
3438,TIC 459959916,Kostov 2022 arXiv:2202.05790,13.323,13.23000,1.054483,8.768239,1.5,1.5,4.4,4.7,15.0,50.0
3416,TIC 13021681,Kostov 2023 arXiv:2309.14200,15.317,14.77170,0.540612,11.065090,1.0,NaN,3.1,3.7,40.0,360.0


In [5]:
#exposure data from Robbie's LBT observing readme, with dupes removed manually
import numpy as np
mag=np.array([7.9,9.4,9.5,10,10.2,10.3,10.6,10.7,10.8,10.9,11,11.1,11.2,11.3,11.4,11.5,11.7,11.8,11.9,12,12.1,12.3,12.4,12.5,12.6,12.7,12.8,13.2,13.3,13.4,13.5,13.6,13.7,13.8,13.9,])
sec=np.array([60,60,60,60,60,60,60,60,60,72,72,84,78,84,102,102,132,138,150,192,180,240,270,258,270,318,360,510,594,630,666,726,768,858,918,])

from scipy.interpolate import CubicSpline
cs = CubicSpline(mag, sec)

#print out results in LBT readme format
#NOTE: targets are printed in ascending RA.  Rotate the list so first observable target is first.
print("TargetName       RA (J2000)     DEC (J2000)       Gmag     pmRA     pmDEC     Mode               ExposureTime   ExecutionTime        Priority  Notes")
print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
for _,target in targets[targets["local_id"].isin(combined_ids)].sort_values("ra").iterrows():
    exposure = cs(target["magnitude"])
    readout_sec = exposure + 140
    readout_min = readout_sec / 60
    print(
        f"{target["local_id"]:<15s} "
        f"{Angle(target["ra"]*u.deg).to_string(unit=u.hour,decimal=False,precision=2,sep=":"):>11s}    "
        f"{Angle(target["dec"]*u.deg).to_string(unit=u.deg,decimal=False,precision=2,sep=":",alwayssign=True):>12s}      "
        f"{target["magnitude"]:5.2f}    "
        f"{target["pmra"]:5.2f}    "
        f"{target["pmdec"]:6.2f}     "
        f"F300/CDII/CDVI     "
        f"1x{exposure:4.0f}sec     "
        f"{readout_sec:5.0f} sec = {readout_min:4.1f}min   "
        f"{target["Priority"]:<7s}   "
        "--"
    )

TargetName       RA (J2000)     DEC (J2000)       Gmag     pmRA     pmDEC     Mode               ExposureTime   ExecutionTime        Priority  Notes
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
TIC 284814380    0:47:27.28    +64:49:04.86      12.02    -2.56     -0.24     F300/CDII/CDVI     1x 193sec       333 sec =  5.6min   Medium    --
TIC 307119043    0:59:18.61    +51:13:17.91       9.98     7.56     -4.11     F300/CDII/CDVI     1x  60sec       200 sec =  3.3min   Highest   --
TIC 52856877     1:09:20.23    +61:02:28.48      10.85     3.22     -1.72     F300/CDII/CDVI     1x  66sec       206 sec =  3.4min   High      --
TIC 389836747    1:33:10.91    +61:35:07.14      10.71    -3.38     -4.47     F300/CDII/CDVI     1x  60sec       200 sec =  3.3min   Low       --
TIC 454140642    4:19:05.63     +0:54:00.15      10.22     2.83    -10.85     F30

In [6]:
#output targets in LBT obs.txt format
#NOTE: targets are printed in ascending RA.  Rotate the list so first observable target is first.
print("#TargetName      RA(J2000)      DEC(J2000)        VMag      MODE               t_exp")
for _,target in targets[targets["local_id"].isin(combined_ids)].sort_values("ra").iterrows():
    exposure = cs(target["magnitude"])
    readout_sec = exposure + 140
    readout_min = readout_sec / 60
    print(
        f"{target["local_id"]:<15s}  "
        f"{Angle(target["ra"]*u.deg).to_string(unit=u.hour,decimal=False,precision=2,sep=":"):>11s}    "
        f"{Angle(target["dec"]*u.deg).to_string(unit=u.deg,decimal=False,precision=2,sep=":",alwayssign=True):>12s}      "
        f"{target["magnitude"]:5.2f}     "
        f"F300/CDII/CDVI     "
        f"1x{exposure:4.0f}sec     "
    )

#TargetName      RA(J2000)      DEC(J2000)        VMag      MODE               t_exp
TIC 284814380     0:47:27.28    +64:49:04.86      12.02     F300/CDII/CDVI     1x 193sec     
TIC 307119043     0:59:18.61    +51:13:17.91       9.98     F300/CDII/CDVI     1x  60sec     
TIC 52856877      1:09:20.23    +61:02:28.48      10.85     F300/CDII/CDVI     1x  66sec     
TIC 389836747     1:33:10.91    +61:35:07.14      10.71     F300/CDII/CDVI     1x  60sec     
TIC 454140642     4:19:05.63     +0:54:00.15      10.22     F300/CDII/CDVI     1x  60sec     
TIC 459959916     4:45:57.52     +4:49:46.63      13.23     F300/CDII/CDVI     1x 536sec     
TIC 391620600     4:46:33.69    +44:45:13.18      12.49     F300/CDII/CDVI     1x 259sec     
TIC 367448265     5:13:31.79    +35:39:10.99       7.87     F300/CDII/CDVI     1x  60sec     
TIC 123098844    18:38:17.48    +44:41:54.96      11.11     F300/CDII/CDVI     1x  84sec     
TIC 278352276    20:30:00.87    +48:36:25.40      10.32     F300/CDII